## This notebook shows how to use Orchestrator APIs for user experiments

In [6]:
import os
from fabric_cm.credmgr.credmgr_proxy import CredmgrProxy
from fabric_cf.orchestrator.orchestrator_proxy import OrchestratorProxy
import json

In [7]:
credmgr_host = os.environ['FABRIC_CREDMGR_HOST']
credmgr_proxy = CredmgrProxy(credmgr_host=credmgr_host)
orchestrator_host = os.environ['FABRIC_ORCHESTRATOR_HOST']
orchestrator_proxy = OrchestratorProxy(orchestrator_host=orchestrator_host)

### Fabric Tokens
Fabric has 2 kinds of tokens:<br>
- Identity : required for Control/Measurement Framework APIs. Identity Token is valid upto an hour.
- Refresh : required to generate new Identity Tokens valid. Refresh Token is valid for 24 hours.

Fabric Identity token is required for Control/Measurement Framework APIs.<br>

When user logins to Jupyterhub after authenticating against CILogon, OIDC refresh token is derived. <br>
This token is available as the environment variable `CILOGON_REFRESH_TOKEN`.<br>

On the first login, we use `CILOGON_REFRESH_TOKEN` to generate new Fabric Identity Token and Fabric Refresh Token.<br>
For any subsequent use, we use Fabric Refresh Token. On every refresh, Fabric Refresh Token is changed and updated.<br>

NOTE: These steps are required for any experiments on Fabric Testbed.<br>

In [8]:
not_found=False
fabric_refresh_token=None
%store -r fabric_refresh_token

if fabric_refresh_token is None:
    fabric_refresh_token=os.environ['CILOGON_REFRESH_TOKEN']
    %store fabric_refresh_token
print("Fabric Refresh Token {}".format(fabric_refresh_token))
print("CILOGON_REFRESH_TOKEN environment variable: {}".format(os.environ['CILOGON_REFRESH_TOKEN']))

Fabric Refresh Token https://cilogon.org/oauth2/refreshToken/1368d6e7c8a7f53223d68c44217ec6bf/1614350937681
CILOGON_REFRESH_TOKEN environment variable: https://cilogon.org/oauth2/refreshToken/1368d6e7c8a7f53223d68c44217ec6bf/1614350937681


### Get new Fabric Identity Token and update Fabric Refresh Token

In [9]:
try:
    refresh_res = credmgr_proxy.refresh(project_name='all', scope='all', refresh_token=fabric_refresh_token)
    print("New Tokens received: {}".format(json.dumps(refresh_res)))
    fabric_id_token=refresh_res['id_token']
    fabric_refresh_token=refresh_res['refresh_token']
    print()
    print("New Refresh Token: {}".format(fabric_refresh_token))
    print()
    print("Stored new Refresh Token")
    %store fabric_refresh_token
except Exception as e:
    print("Exception occurred while getting tokens:{}".format(e))

Exception occurred while getting tokens:('INTERNAL SERVER ERROR', '"(\\"server_error\\"\\nerror_description=\\"No transaction found for identifier \\"https://cilogon.org/oauth2/refreshToken/1368d6e7c8a7f53223d68c44217ec6bf/1614350937681\\"\\"\\n) "\n')


### Orchestrator API example to query for available resources

In [10]:
resources = orchestrator_proxy.resources(token=fabric_id_token)

print(resources)

NameError: name 'fabric_id_token' is not defined

##Create Slice

In [ ]:
import fim.user as fu
t = fu.ExperimentTopology()
n1 = t.add_node(name='n1', site='RENC')
cap = fu.Capacities()
cap.set_fields(core=4, ram=64, disk=500)
n1.set_properties(capacities=cap, image_type='qcow2', image_ref='default_centos_8')
n1.add_component(ctype=fu.ComponentType.SmartNIC, model='ConnectX-6', name='nic1')

n2 = t.add_node(name='n2', site='RENC')
n2.set_properties(capacities=cap, image_type='qcow2', image_ref='default_centos_8')
n2.add_component(ctype=fu.ComponentType.GPU, model='RTX6000', name='nic2')

slice_graph = t.serialize()

status, result = orchestrator_proxy.create(token=fabric_id_token, slice_name=name, slice_graph=slice_graph)

print("Response Status {}".format(status))
print("Response received {}".format(result))

slice_id = result['reservations'][0]['slice_id']

## Query Slices

In [ ]:
status, result = orchestrator_proxy.slices(token=fabric_id_token)

print("Response Status {}".format(status))
print("Response received {}".format(result))

## Query Slice

In [ ]:
status, result = orchestrator_proxy.get_slice(token=fabric_id_token, slice_id=slice_id)

print("Response Status {}".format(status))
print("Response received {}".format(result))

## Query Slivers

In [ ]:
status, result = orchestrator_proxy.slivers(token=fabric_id_token, slice_id=slice_id)

print("Response Status {}".format(status))
print("Response received {}".format(result))

sliver_id = result['reservations'][0]['reservation_id']

## Query Sliver

In [ ]:
status, result = orchestrator_proxy.slivers(token=fabric_id_token, slice_id=slice_id, sliver_id=sliver_id)

print("Response Status {}".format(status))
print("Response received {}".format(result))

## Slice Status

In [ ]:
status, result = orchestrator_proxy.slice_status(token=fabric_id_token, slice_id=slice_id)

print("Response Status {}".format(status))
print("Response received {}".format(result))

## Sliver Status

In [ ]:
status, result = orchestrator_proxy.sliver_status(token=fabric_id_token, slice_id=slice_id, sliver_id=sliver_id)

print("Response Status {}".format(status))
print("Response received {}".format(result))

## Delete Slice

In [ ]:
status, result = orchestrator_proxy.delete(token=fabric_id_token, slice_id=slice_id)

print("Response Status {}".format(status))
print("Response received {}".format(result))